In [1]:
%load_ext autoreload
%autoreload 2

Dependencies

In [2]:
import pathlib as pb

# Environment
ROOT_PATH = pb.Path('..')
DATA_DIR_PATH = ROOT_PATH / 'data'
CACHE_DIR_PATH = ROOT_PATH / '.cache'
TRANSFORMERS_CACHE_DIR_PATH = CACHE_DIR_PATH / 'transformers'
DATASETS_CACHE_DIR_PATH = CACHE_DIR_PATH / 'datasets'
TEST_DATA_FILE = DATA_DIR_PATH / 'test_data.csv'
TRAIN_DATA_FILE = DATA_DIR_PATH / 'train_data.csv'

# Model Repositories
BERT_RO_MODEL_REPO = 'dumitrescustefan/bert-base-romanian-cased-v1'

In [14]:

import os

os.environ['TRANSFORMERS_CACHE'] = str(TRANSFORMERS_CACHE_DIR_PATH)
os.environ['HF_DATASETS_CACHE'] = str(DATASETS_CACHE_DIR_PATH)
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
import torchdata
import torchtext
from torch import nn
from torch import optim
from torch.utils.data import Subset, DataLoader, Dataset
from torch import backends
import typing
import pathlib as pb
import os
import gc
from typing import List, Tuple, Dict, Set
import random
import numpy as np
import pandas as pd
import platform
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import KFold

/home/invokariman/projects/git/nitro-lang-processing-2/.cache/pypoetry/virtualenvs/nitro-lang-processing-2-NQx-kwHa-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from utils import get_available_device, read_data
from preprocess import BertPreprocessor
from data import SexismDataset
from models import BertFlatClassModel

Environment Setup

In [20]:
# Use available GPU
DEVICE: torch.device = get_available_device()

# Deterministic experiments
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
np.random.RandomState(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

Dataset & Preprocessing

In [21]:
train_data_raw, test_data_raw = read_data(DATA_DIR_PATH)

In [22]:
train_dataset = SexismDataset(train_data_raw, BertPreprocessor(train_data_raw, BERT_RO_MODEL_REPO))
test_dataset = SexismDataset(test_data_raw, BertPreprocessor(test_data_raw, BERT_RO_MODEL_REPO))

Model Training

In [23]:
# Training specifications
batch_size = 8
num_workers = 6
learning_rate = 1e-5

In [24]:
# Instantiate language model
# Inspired from: https://luv-bansal.medium.com/fine-tuning-bert-for-text-classification-in-pytorch-503d97342db2
model = BertFlatClassModel(BERT_RO_MODEL_REPO)
loss_fn = nn.CrossEntropyLoss() # TODO: weighted loss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Some weights of the model checkpoint at dumitrescustefan/bert-base-romanian-cased-v1 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
list(model.named_parameters())[0]

('bert_model.embeddings.word_embeddings.weight',
 Parameter containing:
 tensor([[ 1.0032e-02, -7.7607e-02, -5.1038e-02,  ..., -5.6072e-03,
          -1.0463e-02, -1.2911e-02],
         [ 1.1192e-02,  2.2556e-02,  2.0151e-02,  ...,  3.6971e-02,
          -5.8294e-02,  7.2390e-02],
         [ 3.3332e-02, -2.4780e-02, -1.8383e-02,  ...,  3.8183e-03,
          -4.3072e-03, -1.1747e-02],
         ...,
         [-3.1083e-02, -8.0622e-02, -5.6654e-03,  ..., -7.0019e-02,
           1.0483e-02,  5.3974e-02],
         [-1.1946e-02, -2.3286e-02, -4.0363e-02,  ..., -6.9447e-02,
           1.9006e-02, -3.6833e-05],
         [-5.2065e-02, -7.5197e-02, -4.3944e-02,  ..., -5.6147e-02,
           5.3486e-02, -7.2240e-03]], requires_grad=True))

In [28]:
# Inspired from: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

for i, (train_idx, valid_idx) in enumerate(kf.split(train_dataset)):
    # Split the data
    train_subset = Subset(train_dataset, train_idx)
    valid_subset = Subset(train_dataset, valid_idx)

    # Create the dataloaders
    train_loader = DataLoader(
        train_subset,
        batch_size,
        pin_memory=True,
        num_workers=num_workers
    )
    valid_loader = DataLoader(
        train_subset,
        batch_size,
        pin_memory=True,
        num_workers=num_workers
    )

In [30]:
x = next(iter(train_loader))
x

{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'input_ids': tensor([[    2,    36, 17439,  ...,     0,     0,     0],
         [    2,    36,  3106,  ...,     0,     0,     0],
         [    2,    36, 17776,  ...,     0,     0,     0],
         ...,
         [    2,    36, 22237,  ...,     0,     0,     0],
         [    2,    36, 16002,  ...,     0,     0,     0],
         [    2, 43960, 12901,  ...,     0,     0,     0]]),
 'label': tensor([[3, 3, 3,  ..., 3, 3, 4],
         [3, 3, 3,  ..., 3, 3, 4],
         [3, 3, 3,  ..., 3, 3, 4],
         ...,
         [3, 3, 3,  ..., 3, 3, 4],
         [3, 3, 3,  ..., 3, 3, 4],
         [3, 3, 3,  ..., 3, 3, 4]])}

In [ ]:
# Free memory
del model
gc.collect()
torch.cuda.empty_cache()